<a href="https://colab.research.google.com/github/Prakhosha/CNN-seed-detection/blob/master/CNN_seed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Angga Bayu Prakhosha - 13316023

# **Convolutional Neural Network pada pendeteksi benih tanaman**

Dalam notebook ini akan dilakukan pengelompokkan benih tanaman ke dalam 12 kelas menggunakan CNN. Tujuan dari projek ini adalah:

1. Membandingkan model CNN dengan transfer learning dibanding tanpa transfer learning
2. Membandingkan model CNN dengan augmentasi vs dibanding tanpa augmentasi

Untuk mengetahui hal tersebut dilakukan training menggunakan tiga model dengan rincian:

1. Model dengan Transfer Learning dan dengan augmentasi
2. Model dengan Transfer Learning tanpa augmentasi
3. Model tanpa Transfer Learning namun menggunakan augmentasi

## Preproses

In [ ]:
#Import library yang dibutuhkan
import os
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Flatten, Dense, AveragePooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [ ]:
#melihat apakah GPU aktif
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
# mount gdrive, karena menggunakan Google Colab
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/My Drive/Siscer/Tugas3

/content/gdrive/My Drive/Siscer/Tugas3


In [ ]:
#definisi letak direktori data training dan test, serta nama kelas
train_dir = 'train-224x224'
test_dir = 'test-224x224'

classnames=['Black-grass','Charlock','Cleavers','Common Chickweed','Common wheat','Fat Hen','Loose Silky-bent','Maize','Scentless Mayweed','Shepherds Purse','Small-flowered Cranesbill','Sugar beet']


## Training

### Dengan Transfer Learning dan Augmentasi

In [ ]:

img_width = 224
img_height = 224
learning_rate = 0.0001   
logfile = '-train_TL_AUG' + '.log'   
batch_size=32
nbr_epochs=30

#Augmentasi
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,  
    width_shift_range=0.2,  
    height_shift_range=0.2,  
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    classes=classnames, 
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    classes=classnames, 
    class_mode='categorical',
    subset='validation') # set as validation data

Found 3812 images belonging to 12 classes.
Found 949 images belonging to 12 classes.


In [ ]:
cnn_notop=keras.applications.nasnet.NASNetMobile(input_shape=(img_width, img_height, 3)
                                                 , include_top=False
                                                 , weights='imagenet' # menggunakan transfer learning
#                                               , weights=None # tanpa transfer learning
                                                 , input_tensor=None
                                                 , pooling=None)

x = cnn_notop.output  
x = GlobalAveragePooling2D()(x)      
x = Dense(1024, activation='relu')(x)      
x = BatchNormalization()(x)  
predictions = Dense(12, activation='softmax')(x)  
the_model = Model(cnn_notop.input, predictions)

19996672/19993432 [==============================] - 1s 0us/step


In [ ]:
optimizer = SGD(lr=learning_rate, momentum=0.9, decay=0.0, nesterov=True)  
the_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])  
csv_logger = CSVLogger(logfile, append=True)  
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')  
best_model_filename='transferlearning-aug-weights.{epoch:02d}-{val_loss:.2f}.h5'   
best_model = ModelCheckpoint(best_model_filename, monitor='val_accuracy', verbose=1, save_best_only=True)

In [ ]:
the_model.summary()

In [ ]:
the_model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    verbose=1,
    epochs = nbr_epochs,
    callbacks=[best_model, csv_logger, early_stopping])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
119/119 [==============================] - ETA: 0s - loss: 2.4666 - accuracy: 0.2295 
Epoch 00001: val_accuracy improved from -inf to 0.19073, saving model to transferlearning-aug-weights.01-2.31.h5
119/119 [==============================] - 2445s 21s/step - loss: 2.4666 - accuracy: 0.2295 - val_loss: 2.3051 - val_accuracy: 0.1907
Epoch 2/30
119/119 [==============================] - ETA: 0s - loss: 1.6384 - accuracy: 0.4659
Epoch 00002: val_accuracy improved from 0.19073 to 0.27802, saving model to transferlearning-aug-weights.02-2.15.h5
119/119 [==============================] - 93s 778ms/step - loss: 1.6384 - accuracy: 0.4659 - val_loss: 2.1538 - val_accuracy: 0.2780
Epoch 3/30
119/119 [==============================] - ETA: 0s - loss: 1.2260 - accuracy: 0.6032
Epoch 00003: val_accuracy improved from 0.27802 to 0.35237, saving model to transferlearning-aug-weights.03-2.10.h5
119/119 [=============

### Dengan Transfer Learning Tanpa Augmentasi

In [ ]:

img_width = 224
img_height = 224
learning_rate = 0.0001   
logfile = '-train_TL_noAUG' + '.log'   
batch_size=32
nbr_epochs=30

#Augmentasi
train_datagen = ImageDataGenerator(rescale=1./255,
    #shear_range=0.2,
    #zoom_range=0.2,
    #horizontal_flip=True,
    #vertical_flip=True,
    #rotation_range=90,  
    #width_shift_range=0.2,  
    #height_shift_range=0.2,  
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    classes=classnames, 
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    classes=classnames, 
    class_mode='categorical',
    subset='validation') # set as validation data

Found 3812 images belonging to 12 classes.
Found 949 images belonging to 12 classes.


In [ ]:
cnn_notop=keras.applications.nasnet.NASNetMobile(input_shape=(img_width, img_height, 3)
                                                 , include_top=False
                                                 , weights='imagenet' # menggunakan transfer learning
#                                               , weights=None # tanpa transfer learning
                                                 , input_tensor=None
                                                 , pooling=None)

x = cnn_notop.output  
x = GlobalAveragePooling2D()(x)      
x = Dense(1024, activation='relu')(x)      
x = BatchNormalization()(x)  
predictions = Dense(12, activation='softmax')(x)  
the_model = Model(cnn_notop.input, predictions)

In [ ]:
optimizer = SGD(lr=learning_rate, momentum=0.9, decay=0.0, nesterov=True)  
the_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])  
csv_logger = CSVLogger(logfile, append=True)  
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')  
best_model_filename='transferlearning-noaug-weights.{epoch:02d}-{val_loss:.2f}.h5'   
best_model = ModelCheckpoint(best_model_filename, monitor='val_accuracy', verbose=1, save_best_only=True)

In [ ]:
the_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 111, 111, 32) 0           stem_bn1[0][0]                   
____________________________________________________________________________________________

In [ ]:
the_model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    verbose=1,
    epochs = nbr_epochs,
    callbacks=[best_model, csv_logger, early_stopping])

Epoch 1/30
119/119 [==============================] - ETA: 0s - loss: 2.2610 - accuracy: 0.2894
Epoch 00001: val_accuracy improved from -inf to 0.29741, saving model to transferlearning-noaug-weights.01-2.18.h5
119/119 [==============================] - 73s 617ms/step - loss: 2.2610 - accuracy: 0.2894 - val_loss: 2.1761 - val_accuracy: 0.2974
Epoch 2/30
119/119 [==============================] - ETA: 0s - loss: 1.2817 - accuracy: 0.5931
Epoch 00002: val_accuracy improved from 0.29741 to 0.39009, saving model to transferlearning-noaug-weights.02-1.90.h5
119/119 [==============================] - 69s 582ms/step - loss: 1.2817 - accuracy: 0.5931 - val_loss: 1.9014 - val_accuracy: 0.3901
Epoch 3/30
119/119 [==============================] - ETA: 0s - loss: 0.9009 - accuracy: 0.7212
Epoch 00003: val_accuracy improved from 0.39009 to 0.40086, saving model to transferlearning-noaug-weights.03-1.82.h5
119/119 [==============================] - 69s 581ms/step - loss: 0.9009 - accuracy: 0.7212 -

### Tanpa Transfer Learning dengan Augmentasi

In [ ]:

img_width = 224
img_height = 224
learning_rate = 0.0001   
logfile = '-train_noTL_AUG' + '.log'   
batch_size=32
nbr_epochs=30

#Augmentasi
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=90,  
    width_shift_range=0.2,  
    height_shift_range=0.2,  
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    classes=classnames, 
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True,
    classes=classnames, 
    class_mode='categorical',
    subset='validation') # set as validation data

Found 3812 images belonging to 12 classes.
Found 949 images belonging to 12 classes.


In [ ]:
cnn_notop=keras.applications.nasnet.NASNetMobile(input_shape=(img_width, img_height, 3)
                                                 , include_top=False
#                                                 , weights='imagenet' # menggunakan transfer learning
                                                , weights=None # tanpa transfer learning
                                                 , input_tensor=None
                                                 , pooling=None)

x = cnn_notop.output  
x = GlobalAveragePooling2D()(x)      
x = Dense(1024, activation='relu')(x)      
x = BatchNormalization()(x)  
predictions = Dense(12, activation='softmax')(x)  
the_model = Model(cnn_notop.input, predictions)

In [ ]:
optimizer = SGD(lr=learning_rate, momentum=0.9, decay=0.0, nesterov=True)  
the_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])  
csv_logger = CSVLogger(logfile, append=True)  
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')  
best_model_filename='notransferlearning-aug-weights.{epoch:02d}-{val_loss:.2f}.h5'   
best_model = ModelCheckpoint(best_model_filename, monitor='val_accuracy', verbose=1, save_best_only=True)

In [ ]:
the_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_376 (Activation)     (None, 111, 111, 32) 0           stem_bn1[0][0]                   
____________________________________________________________________________________________

In [ ]:
the_model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    verbose=1,
    epochs = nbr_epochs,
    callbacks=[best_model, csv_logger, early_stopping])

Epoch 1/30
119/119 [==============================] - ETA: 0s - loss: 2.3913 - accuracy: 0.2216
Epoch 00001: val_accuracy improved from -inf to 0.04741, saving model to notransferlearning-aug-weights.01-2.52.h5
119/119 [==============================] - 94s 792ms/step - loss: 2.3913 - accuracy: 0.2216 - val_loss: 2.5203 - val_accuracy: 0.0474
Epoch 2/30
119/119 [==============================] - ETA: 0s - loss: 1.8531 - accuracy: 0.3963
Epoch 00002: val_accuracy did not improve from 0.04741
119/119 [==============================] - 87s 733ms/step - loss: 1.8531 - accuracy: 0.3963 - val_loss: 2.6491 - val_accuracy: 0.0474
Epoch 3/30
119/119 [==============================] - ETA: 0s - loss: 1.5834 - accuracy: 0.4759
Epoch 00003: val_accuracy did not improve from 0.04741
119/119 [==============================] - 86s 721ms/step - loss: 1.5834 - accuracy: 0.4759 - val_loss: 2.8504 - val_accuracy: 0.0431
Epoch 4/30
119/119 [==============================] - ETA: 0s - loss: 1.3923 - accura

## Testing

In [ ]:
#prediction   
batch_size=4  
nbr_test_samples=794    
#choose weights file manually   
#weights_path = 'simpleNASNet-weights.10-0.17.h5' # choose file manually, filename may be different  
#weights_path = 'sesi-01-weights.08-0.58.h5'
weights_path = 'notransferlearning-aug-weights.10-3.58.h5'
test_data_dir = 'test-224x224'   

test_datagen = ImageDataGenerator(rescale=1./255)   
test_generator = test_datagen.flow_from_directory(   
    test_dir,   
    target_size=(img_width, img_height),   
    batch_size=batch_size,   
    shuffle = False, # no shuffling, since filenames must match predictions. Shuffling may change file sequence   
    classes = None, #    
    class_mode = None)   
test_image_list = test_generator.filenames   
print('Loading model and weights')   
predict_model = load_model(weights_path)   
#predict_model = the_model

Found 200 images belonging to 1 classes.
Loading model and weights


In [ ]:
predict_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_376 (Activation)     (None, 111, 111, 32) 0           stem_bn1[0][0]                   
____________________________________________________________________________________________

In [ ]:
print('Begin to predict for testing data ...')   
predictions = predict_model.predict(x=test_generator, steps=nbr_test_samples/batch_size,verbose=1)   
np.savetxt('predictions_aug_notl.txt', predictions) # store prediction matrix, for later analysis if necessary

Begin to predict for testing data ...
199/198 [==============================] - 5s 26ms/step


In [ ]:
#submission  
submission_file='submit_aug_notl.csv'   
print('Begin to write submission file:'+submission_file)   
f_submit = open(submission_file, 'w')   
f_submit.write('file,species\n')   
for i, image_name in enumerate(test_image_list):   
  # find maximum prediction of 12  
  max_index=0  
  max_value=0  
  for x in range(0, 12):  
    if(predictions[i][x]>max_value):  
      max_value=predictions[i][x]  
      max_index=x  
  basename=os.path.basename(image_name)   
  prediction_class = classnames[max_index] # get predictions from array     
  f_submit.write('%s,%s\n' % (basename, prediction_class))   
f_submit.close()   
print('Finished write submission file ..')

Begin to write submission file:submit_aug_notl.csv
Finished write submission file ..


## Kesimpulan

Pengelompokkan benih tanaman ke dalam 12 jenis kelas telah dilakukan. Pengelompokkan ini dilakukan menggunakan tiga model:

1. Model dengan Transfer Learning dan dengan augmentasi
2. Model dengan Transfer Learning tanpa augmentasi
3. Model tanpa Transfer Learning namun menggunakan augmentasi

Hal ini dilakukan untuk membedakkan hal berikut:
1. Model dengan Transfer Learning vs Model tanpa Transfer Learning
2. Model dengan augmentasi vs Model tanpa augmentasi

### Hasil menunjukkan:

transfer learning aug: 
loss: 0.2229 - accuracy: 0.9201 - val_loss: 0.5645 - val_accuracy: 0.8114

transfer learning no aug:
loss: 0.0594 - accuracy: 0.9931 - val_loss: 0.6684 - val_accuracy: 0.7834

no transfer learning aug:
loss: 0.9515 - accuracy: 0.6823 - val_loss: 3.6563 - val_accuracy: 0.0474

_**Ketiga model dilatih dengan epoch seumlah 30. Pengaruh penggunaan augmentasi menaikkan akurasi model dari 78.34% menjadi 81.14%. Hal ini dikarenakan augmentasi menjadikan model memperhatikkan kondisi-kondisi dimana data masukkan tidak sesuai data latih, seperti rotasi gambar, perbesaran gambar, dan sebaginya.**_  

_**Pengaruh penggunaan transfer learning sangat besar, yaitu dari akurasi model seumlah 4.474% menjadi 81.14%. Hal ini dikarenakan transfer learning mengambil bobot dari model yang telah dilakukan pelatihan awal sehingga mempercepat waktu model mencapai kondisi optimal.**_
